In [17]:
import pandas as pd
import modin.pandas as mpd

In [ ]:
# Ruta del archivo Excel
file_path = 'data/original/Ubicación actual de las Versiones Restauradas.xlsx'

# Especifica el nombre o índice de la hoja que deseas importar
sheet_name = 'VERSIONES RESTAURADAS'  # También puedes usar el índice de la hoja, por ejemplo: 0

# Cargar la hoja especificada en un DataFrame
df_verificacion = pd.read_excel(file_path, sheet_name=sheet_name)

# Mostrar las primeras filas del DataFrame
print(df_verificacion.head())


In [21]:
import pandas as pd
import os
import re

# Ruta de la carpeta que contiene los archivos CSV
folder_path = 'data/original/nas_csv'

# Crear un diccionario para almacenar los DataFrames
dfs = {}

# Función para convertir el nombre del archivo a un formato válido
def valid_name(name):
    # Reemplazar caracteres no válidos y espacios por guiones bajos
    name = re.sub(r'[^a-zA-Z0-9_]', '_', name)  # Reemplazar caracteres no alfanuméricos por '_'
    name = name.replace(' ', '_')  # Reemplazar espacios por '_'
    return name

# Iterar sobre cada archivo en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # Obtener la ruta completa del archivo
        file_path = os.path.join(folder_path, filename)
        
        # Leer el archivo CSV en un DataFrame
        df = pd.read_csv(file_path)
        
        # Usar el nombre del archivo (sin la extensión) como clave en el diccionario
        df_name = valid_name(os.path.splitext(filename)[0])  # Convertir a un nombre válido
        dfs[df_name] = df

# Ahora dfs contiene todos los DataFrames
# Ejemplo de cómo acceder a un DataFrame específico
# df_especifico = dfs.get('nombre_del_archivo_sin_extension')

# Mostrar los nombres de los DataFrames creados
print(dfs.keys())

dict_keys(['NAS_10', 'restauracion_1'])


In [ ]:
# Ruta del archivo Excel
file_path = 'data/original/Verificación data de los NAS.xlsx'

# Leer todas las hojas del archivo Excel
xls = pd.ExcelFile(file_path)

# Crear un diccionario para almacenar los DataFrames
dfs = {}

# Iterar sobre cada hoja y crear un DataFrame
for sheet_name in xls.sheet_names:
    # Convertir el nombre de la hoja a un formato válido para Python
    valid_name = sheet_name.replace(" ", "_").replace("-", "_").replace(".", "_")
    valid_name = valid_name.replace("(", "").replace(")", "")  # Eliminar paréntesis si existen
    
    # Leer la hoja en un DataFrame
    dfs[valid_name] = pd.read_excel(xls, sheet_name=sheet_name)

# Ahora dfs contiene todos los DataFrames
# Ejemplo de cómo acceder a un DataFrame específico
# df_nas_1 = dfs.get('nombre_valido_de_la_hoja')

# Para mostrar los nombres de los DataFrames creados
print(dfs.keys())


In [ ]:
df_codigos = pd.DataFrame()

df_codigos[['Codigo', 'Resto']] = df_verificacion['Código versión restaurada'].str.extract(r'(^.*\d{6})\s*(.*)')

# Mostrar el nuevo DataFrame con las dos columnas
print(df_codigos[['Codigo', 'Resto']])

In [22]:

# Supongamos que df_codigos ya está definido como un DataFrame de Modin
# Ejemplo de cómo definir df_codigos (asegúrate de usar mpd para Modin)
# df_codigos = mpd.read_excel('ruta/a/tu/archivo.xlsx')

def verificar_existencia(codigo):
    # Iterar sobre todos los DataFrames en el diccionario
    for df_name, df in dfs.items():
        # Comprobar si el código está en cualquier columna del DataFrame
        if codigo in df.values:
            return "si"
    return "no"

# Aplicar la función para crear la nueva columna 'existente' en df_codigos
df_codigos['existente'] = df_codigos['Codigo'].apply(verificar_existencia)

# Mostrar el DataFrame actualizado
print(df_codigos)


               Codigo                                              Resto  \
0     VR F35mm 851242                                       QUÉ HORA ES?   
1     VR F35mm 851304                             A Orillas Del Amazonas   
2     VR F35mm 851303                                  Ángulos De Bogotá   
3     VR F35mm 851011                      Batalla del pantano de Vargas   
4     VR F35mm 851112                         Cali, la Sultana del Valle   
...               ...                                                ...   
1324  VR F16mm 802705                   Yuruparí Cap Cumbia sobre el río   
1325  VR F16mm 802706               Yuruparí Cap Las farotas de Talaigua   
1326  VR F16mm 802707  Yuruparí Cap Mompox, el ocaso del oro y del barro   
1327  VR F16mm 802708               Yuruparí Cap El Cristo negro de Tadó   
1328  VR F16mm 802709                Yuruparí Cap El Carnaval del Diablo   

     existente  
0           no  
1           no  
2           no  
3           no  
4 

In [23]:
# Combinar todos los DataFrames en uno solo
df_todos = pd.concat(dfs.values(), ignore_index=True)

# Crear un conjunto de códigos existentes
codigos_existentes = set(df_todos.values.flatten())

# Aplicar la verificación de existencia
df_codigos['existente'] = df_codigos['Codigo'].isin(codigos_existentes).replace({True: "si", False: "no"})

# Mostrar el DataFrame actualizado
print(df_codigos)
